# Configuracion

### Carga de librerias

In [1]:
from pandas import merge

In [52]:
from sklearn.preprocessing import label_binarize
from pandas import DataFrame

### Definicion de funciones

In [27]:
def integracion1(sets_df, train_o_envio = 'train'):
    col_llave = 'tube_assembly_id'
    df = merge(left = sets_df[train_o_envio], right = sets_df['bill'], how = 'left', on = col_llave)
    df = merge(left = df, right = sets_df['tube'], how = 'left', on = col_llave)
    return df

In [80]:
def limpieza2(df):
    # limpieza de NaN's en las columnas de component_id_n
    return df.dropna(axis = 0, how = 'all', subset = ['component_id_{}'.format(i) for i in range(1, 9)])

In [232]:
from IPython.core.debugger import Tracer
trace = Tracer()

In [456]:
def integracion3(sets_df, df):
    resultado = None
    # Establecer las clases que tendra la vectorizacion
    tipos_comp = sorted(sets_df['components'].component_type_id.unique())
    # establecer los datos de los componentes
    df_tipo_comp = sets_df['components'][['component_id', 'component_type_id']]
    # Iterar para cada columna componente
    for i in range(1, 9):
        # la cantidad y componente correspondientes para la iteracion
        cantidad, componente = 'quantity_{}'.format(str(i)), 'component_id_{}'.format(str(i))
        # Agregar al df de entrenamiento la columna de "tipo de componente" para el componente_i
        entrenamiento = df[[componente]].reset_index().dropna(axis= 0, how= 'all', subset= [componente])
        no_vect = merge(left = entrenamiento, right = df_tipo_comp, how = 'left', left_on = componente,
                        right_on = 'component_id', copy = False).set_index('index')
        # Vectorizar el tipo de componente
        vectorizacion = DataFrame(data= label_binarize(no_vect.component_type_id, classes=tipos),
                                 index= no_vect.index, columns= tipos)
        # Multiplicar la matriz de la vectorizacion por la cantidad solicitada
        vectorizacion = vectorizacion.mul(df.loc[no_vect.index, cantidad], axis = 0)
        # Sumar el producto a la la suma de las matrices de vectorizacion
        #try:
        #    resultado += vectorizacion
        #except UnboundLocalError:
        #    resultado = vectorizacion        
        if resultado is not None:
            resultado = resultado.add(vectorizacion, axis= 0, fill_value= 0)
        else:
            resultado = vectorizacion
        #trace()
    return resultado

In [521]:
def vectorizacion(df, nombCol, eliminarColOriginal = True):
    # limpieza de renglones sin valores
    df_temp = df[[nombCol]].reset_index().dropna(axis= 0, how= 'all', subset= [nombCol]).set_index('index')
    # establecimiento de las clases de la variable nominal
    clases = sorted(df_temp[nombCol].unique())
    # creacion de la binarizacion estandar
    array_binarizado = label_binarize(df_temp[nombCol], classes= clases)
    # dataframe de la binarizacion respetando el indice
    array_binarizado = DataFrame(data= array_binarizado, index= df_temp.index,
                                 columns= ['{}__{}'.format(nombCol, str(clase)) for clase in clases])
    if eliminarColOriginal:
        df = df.drop(labels= nombCol, axis= 1)
    return merge(left= df, right= array_binarizado, how= 'left', left_index= True,
                 right_index= True, copy= False)

# Ejecucion de rutina

In [517]:
from Metadatos import generaPathProyecto
from CargaDatos import retornaSets
path_proyecto = generaPathProyecto()
# sets_df = retornaSets(path_proyecto)
df = integracion1(sets_df = sets_df)
df = limpieza2(df)

In [518]:
df = merge(left= df, right= integracion3(sets_df= sets_df, df= df), how= 'left', left_index= True,
           right_index= True, copy= False)

In [519]:
df.drop(labels= [u'component_id_1', u'quantity_1', u'component_id_2', u'quantity_2', 
                 u'component_id_3', u'quantity_3', u'component_id_4', u'quantity_4',
                 u'component_id_5', u'quantity_5', u'component_id_6', u'quantity_6',
                 u'component_id_7', u'quantity_7', u'component_id_8', u'quantity_8'],
        axis= 1, inplace= True)

In [520]:
df.columns

Index([u'tube_assembly_id', u'supplier', u'quote_date', u'annual_usage',
       u'min_order_quantity', u'bracket_pricing', u'quantity', u'cost',
       u'material_id', u'diameter', u'wall', u'length', u'num_bends',
       u'bend_radius', u'end_a_1x', u'end_a_2x', u'end_x_1x', u'end_x_2x',
       u'end_a', u'end_x', u'num_boss', u'num_bracket', u'other', u'CP-001',
       u'CP-002', u'CP-003', u'CP-004', u'CP-005', u'CP-006', u'CP-007',
       u'CP-008', u'CP-009', u'CP-010', u'CP-011', u'CP-012', u'CP-014',
       u'CP-015', u'CP-016', u'CP-017', u'CP-018', u'CP-019', u'CP-020',
       u'CP-021', u'CP-022', u'CP-023', u'CP-024', u'CP-025', u'CP-026',
       u'CP-027', u'CP-028', u'CP-029', u'OTHER'],
      dtype='object')

In [524]:
df = vectorizacion(df, 'supplier')
df = vectorizacion(df, 'material_id')
df = vectorizacion(df, 'end_a')
df = vectorizacion(df, 'end_x')
df = vectorizacion(df, 'other')

In [525]:
df.shape

(28751, 168)

In [527]:
for columna in df.columns:
    print columna

tube_assembly_id
quote_date
annual_usage
min_order_quantity
bracket_pricing
quantity
cost
diameter
wall
length
num_bends
bend_radius
end_a_1x
end_a_2x
end_x_1x
end_x_2x
num_boss
num_bracket
CP-001
CP-002
CP-003
CP-004
CP-005
CP-006
CP-007
CP-008
CP-009
CP-010
CP-011
CP-012
CP-014
CP-015
CP-016
CP-017
CP-018
CP-019
CP-020
CP-021
CP-022
CP-023
CP-024
CP-025
CP-026
CP-027
CP-028
CP-029
OTHER
supplier__S-0003
supplier__S-0004
supplier__S-0005
supplier__S-0006
supplier__S-0007
supplier__S-0008
supplier__S-0009
supplier__S-0011
supplier__S-0012
supplier__S-0013
supplier__S-0014
supplier__S-0015
supplier__S-0018
supplier__S-0022
supplier__S-0023
supplier__S-0024
supplier__S-0025
supplier__S-0026
supplier__S-0027
supplier__S-0029
supplier__S-0030
supplier__S-0031
supplier__S-0041
supplier__S-0042
supplier__S-0046
supplier__S-0050
supplier__S-0054
supplier__S-0056
supplier__S-0058
supplier__S-0059
supplier__S-0060
supplier__S-0062
supplier__S-0064
supplier__S-0066
supplier__S-0070
supplier__S-0